# ETL para o banco de dados Neo4j

In [1]:
%pip install tqdm

Note: you may need to restart the kernel to use updated packages.


## Importações

In [2]:
import os
import sys
from tqdm import tqdm

import pandas as pd

from pyspark.sql import SparkSession

## Constantes

In [3]:
DATASET_PATH = '../../dataset'

USER_FILTERED_PATH = os.path.join(DATASET_PATH, 'user-filtered.csv')

## Código para resolver um problema de versão encontrada pelo PySpark

A resolução do erro foi encontrada em uma resposta no [StackOverflow](https://stackoverflow.com/a/65010346)

In [4]:
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

## Configurando inicialização do "conector" Neo4j spark

In [5]:
spark = SparkSession.\
        builder.\
        config("spark.jars", "../../spark-neo4j/neo4j-connector-apache-spark_2.12-5.0.1_for_spark_3.jar").\
        getOrCreate()

## Lendo o arquivo user-filtered.csv

In [6]:
df_user_filtred = spark.read.csv(USER_FILTERED_PATH, inferSchema=True, header=True, escape='"', multiLine=True)

In [7]:
df_user_filtred.show()

+-------+--------+------+
|user_id|anime_id|rating|
+-------+--------+------+
|      0|      67|     9|
|      0|    6702|     7|
|      0|     242|    10|
|      0|    4898|     0|
|      0|      21|    10|
|      0|      24|     9|
|      0|    2104|     0|
|      0|    4722|     8|
|      0|    6098|     6|
|      0|    3125|     9|
|      0|     481|    10|
|      0|      68|     6|
|      0|    1689|     6|
|      0|    2913|     6|
|      0|    1250|     7|
|      0|     356|     9|
|      0|     121|     9|
|      0|     430|     9|
|      0|    1829|     7|
|      0|    1571|    10|
+-------+--------+------+
only showing top 20 rows



## Filtrando a base de dados

Para tentar resolver isso, filtramos a base de dados, removendo todas as avaliações abaixo de 8. Como definimos o limiar para 8, os resultados das consultas continuarão iguais. Porém, seria ideal armazenar todas as relações, pois caso esse limiar seja modificado para algum valor inferior, seria necessário adicionar no banco de dados as interações com as notas inferiores. Caso todas as relações fossem armazenadas, bastaria modificar a filtragem para considerar esse novo valor de limiar. Além disso, caso o sistema de recomendações fosse modificado, para usar tanto as avaliações negativas como negativas para gerar recomendações, seria necessário ter todas as avaliações no banco de dados.

In [8]:
df_user_filtred = df_user_filtred.filter(df_user_filtred.rating >= 8)
display(df_user_filtred.show())
df_user_filtred.count()

+-------+--------+------+
|user_id|anime_id|rating|
+-------+--------+------+
|      0|      67|     9|
|      0|     242|    10|
|      0|      21|    10|
|      0|      24|     9|
|      0|    4722|     8|
|      0|    3125|     9|
|      0|     481|    10|
|      0|     356|     9|
|      0|     121|     9|
|      0|     430|     9|
|      0|    1571|    10|
|      0|     578|    10|
|      0|     431|     8|
|      0|    2762|     9|
|      0|    3418|     9|
|      0|    2034|     8|
|      0|     164|     8|
|      0|     459|     9|
|      0|     419|     8|
|      0|     199|     8|
+-------+--------+------+
only showing top 20 rows



None

32802476

In [ ]:
df_user_filtred = df_user_filtred.limit(1000000)

## Salvando os dados no banco de dados

### Salvando os nós de usuários

In [9]:
display(df_user_filtred.select('user_id').distinct().show())

+-------+
|user_id|
+-------+
|    148|
|    463|
|    496|
|    833|
|   1088|
|   1342|
|   1580|
|   1591|
|   1645|
|   1959|
|   2122|
|   2142|
|   2366|
|   2659|
|   2866|
|   3175|
|   3749|
|   3794|
|   3997|
|   4101|
+-------+
only showing top 20 rows



None

In [10]:
df_user_filtred.select('user_id').distinct().write \
  .format("org.neo4j.spark.DataSource") \
  .mode("Append") \
  .option("url", "bolt://localhost:7687") \
  .option("labels", ":User") \
  .save()

### Salvando os nós de animes

In [11]:
display(df_user_filtred.select('anime_id').distinct().show())

+--------+
|anime_id|
+--------+
|    5300|
|   30654|
|   38422|
|     496|
|    8086|
|    1088|
|    6336|
|     463|
|    2142|
|    9465|
|    3918|
|   25517|
|   17389|
|   40515|
|    1829|
|   22097|
|   33569|
|    1591|
|     148|
|   36538|
+--------+
only showing top 20 rows



None

In [12]:
df_user_filtred.select('anime_id').distinct().write \
  .format("org.neo4j.spark.DataSource") \
  .mode("Append") \
  .option("url", "bolt://localhost:7687") \
  .option("labels", ":Anime") \
  .save()

### Salvando as relações entre usuários e animes

In [14]:
df_user_filtred.write.format("org.neo4j.spark.DataSource")\
    .mode("Append")\
    .option("url", "neo4j://localhost:7687")\
    .option("relationship", "Rating")\
    .option("relationship.save.strategy", "keys")\
    .option("relationship.source.labels", ":User")\
    .option("relationship.source.save.mode", "overwrite")\
    .option("relationship.source.node.keys", "user_id:user_id")\
    .option("relationship.target.labels", ":Anime")\
    .option("relationship.target.save.mode", "overwrite")\
    .option("relationship.target.node.keys", "anime_id:anime_id")\
    .option("relationship.properties", "rating")\
    .save()